## Extracting Titanic disaster dataset from Kaggle

In [2]:
!pip install python-dotenv

In [4]:
from dotenv import load_dotenv, find_dotenv

In [5]:
# search for .env in all directories
dotenv_path = find_dotenv()
#Load env file entries to the environment variable
load_dotenv(dotenv_path)

True

In [6]:
# Extracting environment variable using os.environ.get
import os
KAGGLE_USERNAME = os.environ.get("KAGGLE_USERNAME")
print (KAGGLE_USERNAME)

mahendermahi77


In [7]:
# imports
import requests
from requests import session
import os
from dotenv import load_dotenv, find_dotenv

In [8]:
# payload for post
payload = {
    'action' : 'login'
    'username' : os.environ.get("KAGGLE_USERNAME")
    'password' : os.environ.get("KAGGLE_PASSWORD")
}

# URL for downloading the dataset
url="https://www.kaggle.com/c/titanic/download/train.csv"

# Session
with session() as c:
    # post request to Kaggle
    c.post("https://www.kaggle.com/account/login",data=payload)
    # get request
    response = c.get(url)
    print (response.text)

SyntaxError: invalid syntax (<ipython-input-8-9f30ff99d127>, line 4)

In [19]:
!pip install --upgrade kaggle

  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73053 sha256=6daaa567a1812f724f4e5bd53bfee9a063772377f0acebbda4181aa544e2bd08
  Stored in directory: c:\users\mahethot\appdata\local\pip\cache\wheels\29\da\11\144cc25aebdaeb4931b231e25fd34b394e6a5725cbb2f50106
Successfully built kaggle


In [ ]:
from IPython.display import display_html
def restartkernel():
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
restartkernel()

### Read API Key

In [9]:
import json
import os

api_file_path = os.path.join(os.path.pardir,'kaggle.json')
with open(api_file_path) as f:
    kaggle_token = json.load(f)
    # kaggle authentication
    os.environ["KAGGLE_USERNAME"] = kaggle_token['username']
    os.environ["KAGGLE_KEY"] = kaggle_token['key']

In [10]:
from kaggle.api.kaggle_api_extended import KaggleApi

In [11]:
# create kaggle API object
api = KaggleApi()
# authenticate
api.authenticate()
# raw data paths
raw_data_path = os.path.join(os.path.pardir,'data','raw')
# download files to the ..data/raw folder
api.competition_download_file(competition = 'titanic',file_name = 'train.csv', path = raw_data_path ,force = True)
api.competition_download_file(competition = 'titanic',file_name = 'test.csv', path = raw_data_path, force = True)

100%|██████████████████████████████████████████████████████████████████████████████| 59.8k/59.8k [00:00<00:00, 436kB/s]


100%|█████████████████████████████████████████████████████████████████████████████| 28.0k/28.0k [00:00<00:00, 1.52MB/s]

In [12]:
train_data_path = os.path.join(raw_data_path, 'train.csv')
!head -5 $train_data_path

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S


In [13]:
!ls -l ../data/raw

total 88
-rw-r--r-- 1 mahethot Domain Users 28629 Jun  5 00:52 test.csv
-rw-r--r-- 1 mahethot Domain Users 61194 Jun  5 00:52 train.csv


#### Building a script file 

In [14]:
get_raw_data_script_file = os.path.join(os.path.pardir,'src','data','get_raw_data.py')

In [17]:
%%writefile $get_raw_data_script_file

import json
import os
import logging

# Root directory
project_dir = os.path.join(os.path.dirname(__file__),os.pardir,os.pardir)
# read API token and create Env variable
api_file_path = os.path.join(project_dir, 'kaggle.json')
with open(api_file_path) as f:
    kaggle_token = json.load(f)
    # Env variables
    os.environ["KAGGLE_USERNAME"] = kaggle_token['username']
    os.environ["KAGGLE_PASSWORD"] = kaggle_token['key']
    
from kaggle.api.kaggle_api_extended import KaggleApi

def main(project_dir):
    '''
    main method
    '''
    
    # get logger
    logger = logging.getLogger(__name__)
    logger.info('geting raw data')
    
    #file name
    train_file_name = 'train.csv'
    test_file_name = 'test.csv'
    
    # file paths
    raw_data_path = os.path.join(project_dir,'data','raw')
    
    # extract data
    api = KaggleApi()
    api.authenticate()
    api.competition_download_file(competition = 'titanic', file_name = train_file_name, path = raw_data_path, force = True)
    api.competition_download_file(competition = 'titanic', file_name = test_file_name , path = raw_data_path, force = True)
    logger.info('downloaded raw training and test data')
    
    
if __name__ == '__main__' :
    # setup logger
    log_fmt = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    logging.basicConfig(level=logging.INFO, format=log_fmt)
    
main(project_dir)

Overwriting ..\src\data\get_raw_data.py


In [18]:
!python $get_raw_data_script_file

2021-06-05 01:29:10,806 - __main__ - INFO - geting raw data
2021-06-05 01:29:31,865 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000277F961B940>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond')': /api/v1/competitions/data/download/titanic/train.csv
2021-06-05 01:29:31,865 - urllib3.connectionpool - WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000277F961B940>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or e